In [1]:
from pathlib import Path
import pickle as pkl
import itertools as it

import pandas as pd

import sys; sys.path.append('..')
import config as cfg

%load_ext autoreload
%autoreload 2

In [2]:
cfg.de_input_dir.mkdir(parents=True, exist_ok=True)
cfg.de_output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
# load gene expression counts data
all_counts_df = pd.read_csv(cfg.processed_counts_file, sep='\t', index_col=0)
print(all_counts_df.shape)
all_counts_df.iloc[:5, :5]

(10530, 60498)


ENSG00000000003.14  ENSG00000000005.5  ENSG00000000419.12  \
sample_id                                                                    
TCGA-02-0047-01                2132                  1                 552   
TCGA-02-0055-01                2590                742                2110   
TCGA-02-2483-01                4438                  1                2230   
TCGA-02-2485-01                4615                  4                1022   
TCGA-04-1331-01                6159                 10                2752   

                 ENSG00000000457.13  ENSG00000000460.16  
sample_id                                                
TCGA-02-0047-01                 387                 269  
TCGA-02-0055-01                 493                 452  
TCGA-02-2483-01                 735                 653  
TCGA-02-2485-01                 629                 672  
TCGA-04-1331-01                1173                1091

In [4]:
# load park et al. initial analysis results here
park_info_df = pd.read_csv(
    cfg.park_info_file, sep='\t'
)

print(park_info_df.shape)
park_info_df.head()

(308148, 7)


sample_id  positive_prob  identifier  mutation_status  copy_status  \
0  TCGA-4E-A92E-01   1.558183e-08  ACVR1_UCEC                0          0.0   
1  TCGA-A5-A0GB-01  2.734236e-237  ACVR1_UCEC                1          0.0   
2  TCGA-A5-A0GJ-01   0.000000e+00  ACVR1_UCEC                0          0.0   
3  TCGA-A5-A0GM-01   0.000000e+00  ACVR1_UCEC                0          0.0   
4  TCGA-A5-A0GW-01   0.000000e+00  ACVR1_UCEC                0          0.0   

  status    class  
0   none  class 1  
1    one  class 1  
2   none  class 1  
3   none  class 1  
4   none  class 1

In [5]:
# for now, we'll just run these with a few hand-picked examples
# in the future we should run for all examples + do some kind of synthesis
class_1_id = 'NF2_KIRP'
class_2_id = 'PTEN_UCEC'
class_3_id = 'KRAS_COADREAD'
class_4_id = 'TP53_BRCA'

class_ids = [class_1_id, class_2_id, class_3_id, class_4_id]

In [6]:
exp_output = str(cfg.de_input_dir / '{}_{}_{}_counts.tsv')
info_output = str(cfg.de_input_dir / '{}_{}_{}_info.tsv')

for class_id in class_ids:
    gene, cancer_type = class_id.split('_')
    for status_1, status_2 in it.combinations(
        ['none', 'one', 'both'], 2
    ):
        
        print(class_id, status_1, status_2)
        
        info_df = park_info_df[
            ((park_info_df.identifier == class_id) &
             (park_info_df.status.isin([status_1, status_2])))
        ].set_index('sample_id')
        
        samples = (
            info_df.index.intersection(all_counts_df.index)
        )
        exp_df = all_counts_df.loc[samples, :]
        info_df = info_df.loc[samples, :]
        
        exp_df.to_csv(
            exp_output.format(class_id, status_1, status_2),
            sep='\t'
        )
        info_df.status.to_csv(
            info_output.format(class_id, status_1, status_2),
            sep='\t'
        )

NF2_KIRP none one
NF2_KIRP none both
NF2_KIRP one both
PTEN_UCEC none one
PTEN_UCEC none both
PTEN_UCEC one both
KRAS_COADREAD none one
KRAS_COADREAD none both
KRAS_COADREAD one both
TP53_BRCA none one
TP53_BRCA none both
TP53_BRCA one both


In [7]:
base_dir = str(cfg.de_base_dir)
input_dir = str(cfg.de_input_dir)
output_dir = str(cfg.de_output_dir)

In [8]:
%load_ext rpy2.ipython

In [9]:
%%R -i base_dir -i input_dir -i output_dir 

source(paste0(base_dir, '/de_analysis.R'))

# this stuff probably shouldn't be hardcoded, but fine for now
identifiers <- c(
    'NF2_KIRP',
    'PTEN_UCEC',
    'KRAS_COADREAD',
    'TP53_BRCA'
)

status_combs <- list()
status_combs[[1]] <- c('none', 'one')
status_combs[[2]] <- c('none', 'both')
status_combs[[3]] <- c('one', 'both')

for (i in 1:length(identifiers)) {
    identifier <- identifiers[i]
    for (j in 1:length(status_combs)) {
        status_1 <- status_combs[[j]][1]
        status_2 <- status_combs[[j]][2]
        counts_file <- paste(input_dir,
                             '/',
                             identifier,
                             '_',
                             status_1,
                             '_',
                             status_2,
                             '_counts.tsv',
                             sep='')
        info_file <- paste(input_dir,
                           '/',
                           identifier,
                           '_',
                           status_1,
                           '_',
                           status_2,
                           '_info.tsv',
                           sep='')
        print(counts_file)
        print(info_file)
        # get_DE_stats_DESeq(info_file,
        #                    counts_file,
        #                    paste(identifier, '_', status_1, '_', status_2,
        #                          sep=''),
        #                    output_dir)
        
    }
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, 

[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_none_one_counts.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_none_one_info.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_none_both_counts.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_none_both_info.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_one_both_counts.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/NF2_KIRP_one_both_info.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/PTEN_UCEC_none_one_counts.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/PTEN_UCEC_none_one_info.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/PTEN_UCEC_none_both_counts.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/PTEN_UCEC_none_both_info.tsv"
[1] "/home/jake/research/mutation-fn/4_de_analysis/de_input/PTEN_UCEC_one_both_counts.tsv"
[1] "/home/